In [ ]:
import pytesseract
from pytesseract import Output
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# show at least 100 rows
pd.set_option('display.max_rows', 100)
your_path_to_png = 'media/<...>'
img = cv2.imread(your_path_to_png)
# tesseract to pandas
d_tess = pytesseract.image_to_data(
    img,
    output_type=Output.DATAFRAME,
    lang='eng',
    config='dpi=150'
)

In [ ]:
d = d_tess.copy()
# change encoding to utf-8
d.text = d.text.str.encode('utf-8').str.decode('utf-8')
# filter out the rows with no text
d = d[(d.conf != -1) & (d.text != ' ')]
# get the bounding box of the text
d = d[['level', 'left', 'top', 'width', 'height', 'conf', 'text']]
# keep only the text with confidence
d = d[d.conf > 70]

In [ ]:
img_anno = img.copy()
for i, row in d.iterrows():
    x = row.left
    y = row.top
    w = row.width
    h = row.height
    alpha = row.conf
    # draw the bounding box
    green = (0, 255, 0)
    green_rectange = np.zeros((h, w, 3), np.uint8)
    green_rectange[:] = green
    img_anno[y:y+h, x:x+w] = cv2.addWeighted(img_anno[y:y+h, x:x+w], 1, green_rectange, alpha/100, 0)
    

plt.figure(figsize=(10,10))
plt.imshow(img_anno)
# add the text
for i, row in d.iterrows():
    x = row.left
    y = row.top
    w = row.width
    h = row.height
    text = row.text
    # move the text above the bounding box
    y = y - 10
    plt.text(x+w/2, y+h/2, text, fontsize=5, ha='center', va='center', color='black')

plt.show()

# Multiple images

In [ ]:
# glob all the images with:
import glob
your_path_with_glob = 'media/<...>/*-page-3.png'
image_paths = glob.glob(your_path_with_glob)
image_paths.sort()
# combine all the image_paths to a string with \n
image_paths = '\n'.join(image_paths)

In [ ]:
# save the image_paths to a file
# format:
# path/to/1.png

# path/to/2.png

# path/to/3.tiff
# create the file if it doesn't exist
from pathlib import Path
Path('image_paths.txt').touch()
with open('image_paths.txt', 'w') as f:
    for image_path in image_paths:
        f.write(image_path + '\n')

In [ ]:
# run a subprocess to get the text from the images
# tesseract image_paths.txt tessOutput.txt

import subprocess

result = subprocess.run(
    ['tesseract', '-', '-', 'tsv'], 
    input=image_paths,
    capture_output=True, 
    text=True, 
    encoding="utf8"
)
result.stdout